In [2]:
# Import statements
from bs4 import BeautifulSoup
import requests
import csv

# Read TSV File
og_tsv=open('bik_dataset.tsv', encoding='ISO-8859-1')
read_tsv= csv.reader(og_tsv, delimiter='\t')

# Create Dictionary with DOI as Key and list of authors as value
# Does not include header and ignores blank values from read function

doi_all_auth_dict={} # dictionary for all authors

doi_first_auth_dict={} # dictionary for just first author

for row in read_tsv:
    if row[3] != "" and row[3] != "DOI":
        all_auth=list(row[0].split(","))
        first_auth= all_auth[0]
        doi_all_auth_dict[row[3]]= all_auth
        doi_first_auth_dict[row[3]]=first_auth

affiliations = []

#grab the affiliation for the first author (This is just for one right now as I was testing)
for doi in doi_first_auth_dict.keys():
    article = "https://doi.org/" + str(doi)
    content = requests.get(article)
    soup = BeautifulSoup(content.content, 'html.parser')
    #print(soup)
    affiliation = soup.find_all("p", id="authAffiliations-0");
    
    for p in affiliation:
        university = p.get_text();
        affiliations.append(university[16:114])
        #university = p.get("span");
        #print(university.strip())
        #print(university)
        break
    break
        
  

    
    
    
print(affiliations)
    
#for k,v in doi_first_auth_dict.items():
    #print("KEY:", k, "     ", "VALUE:", v)
    #break
          
#Example print        
#for i in doi_first_auth_dict:
    #print(doi_first_auth_dict[i])
#for i in doi_all_auth_dict:
    #print(doi_all_auth_dict[i])



['Institute of Pharmacology, Toxicology and Pharmacy, Ludwig-Maximilians-University, Munich, Germany']


In [3]:
#Scrape ResearchGate

#Can get pub rate, lab, size and journals from researchgate


#Login to ResearchGate in order to get info
#username="perkinsm@usc.edu"
username="melissa727p@berkeley.edu"
password="pass123!"

session = requests.Session()
login = session.get("https://www.researchgate.net/application.Login.html")
request = BeautifulSoup(login.text).form.find("input",{"name":"request_token"}).attrs["value"]
params = {"request_token":request,
          "invalidPasswordCount":"0",
          'login': username, 
          'password': password,
          "setLoginCookie":"yes"
          }
session.post("https://www.researchgate.net/application.Login.html", data = params)


#Iterate through each doi
doi_pub_dict={}
doi_labsize={}
doi_journals={}
count=0
for doi in doi_first_auth_dict.keys():
    while count<1:
        doi="10.1371/journal.pone.0053940"
        #adjust to handle a few special cases
        if "Ð" in doi:
            doi=doi.replace("Ð","-") 
        elif doi == 'PMID: 8675338':
            doi = "10.1128/iai.64.6.2282-2287.1996"
        elif doi == 'PMID: 9864199 ':
            doi = "10.1128/IAI.67.1.80-87.1999"
        elif doi == 'PMID: 9620397 ':
            doi = "10.1128/JCM.36.6.1666-1673.1998"
            

        #open article
        article="https://www.researchgate.net/search.Search.html?type=publication&query="+str(doi)
        cont = session.get(article)
        soup = BeautifulSoup(cont.content, 'html.parser')

        #Get ResearchGate links for each author's profile
        authlinks=[]
        for link in soup.find_all("a", {"class":"nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-bare research-detail-author"}):
            authlinks.append((link.get_text(),"https://www.researchgate.net/" + str(link.get('href'))))
        
        #Extract Publication Rate from each profile
        pub_rate={}
        labsize={}
        all_journals={}
        for profile in authlinks:
            #
            content=session.get(profile[1])
            soup = BeautifulSoup(content.content, 'html.parser')
            
            #For those without an account
            authordetails=soup.find_all("h2", {"class":"nova-legacy-e-text nova-legacy-e-text--size-m nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit"})
            
            #For those with an account
            if authordetails==[]:
                authordetails=soup.find_all("div", {"class":"nova-legacy-e-text nova-legacy-e-text--size-xl nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit"})
                counter=0
                for item in authordetails:
                    if counter<1:
                        pub_rate[profile[0]]=item.get_text()
                        counter+=1
                
                
                #Find Lab Members and append to lab_size dictionary
                lab=soup.find_all("div", {"class":"nova-legacy-e-text nova-legacy-e-text--size-s nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-xxs nova-legacy-e-text--color-grey-400"})
                for l in lab:
                    labmem=l.get_text()
                    if "Lab members" in labmem:
                        labmem= labmem[13:].replace(")","")
                    labsize[profile[0]]=labmem
                    
                    
                #Other Journals for those with account
                journ_cont=session.get(profile[1]+"/publications")
                jsoup = BeautifulSoup(journ_cont.content, 'html.parser')
                journ=jsoup.find_all("li", {"class":"nova-legacy-e-list__item nova-legacy-v-publication-item__meta-data-item"})
                months= ["Jan ", "Feb ", "Mar ", "Apr ", "May ", "Jun ", "Jul ", "Aug ", "Sep ", "Oct ", "Nov ", "Dec "]
                journal_lst=[]
                for j in jdetails:
                    journal=j.get_text()
                    #print(journal)

                    true_j = any(months in journal for months in months)
                    
                    if true_j==False and journal not in journal_lst:
                        journal_lst.append(journal)
                        
                all_journals[profile[0]]=journal_lst
            
                    
                #Append items to pub_rate dicitionary          
                for item in authordetails:
                    if "Publications" in item.get_text():
                        itemtxt=item.get_text()[14:]
                        itemtxt=itemtxt.replace(")","")
                        pub_rate[profile[0]]=(itemtxt)
            
            #For those without accounts
            else:
                #Append items to pub_rate dicitionary          
                for item in authordetails:
                    if "Publications" in item.get_text():
                        itemtxt=item.get_text()[14:]
                        itemtxt=itemtxt.replace(")","")
                        pub_rate[profile[0]]=(itemtxt)
                    
                #Other Journals for those without account
                months= ["Jan ", "Feb ", "Mar ", "Apr ", "May ", "Jun ", "Jul ", "Aug ", "Sep ", "Oct ", "Nov ", "Dec "]
                journal_lst=[]
                jdetails=soup.find_all("li", {"class":"nova-legacy-e-list__item publication-item-meta-items__meta-data-item"})
                
                for j in jdetails:
                    journal=j.get_text()
                    #print(journal)

                    true_j = any(months in journal for months in months)
                    
                    if true_j==False and journal not in journal_lst:
                        journal_lst.append(journal)
                        
                all_journals[profile[0]]=journal_lst
            
                    
            

        count+=1
        doi_pub_dict[doi]=pub_rate
        doi_labsize[doi]=labsize
        doi_journals[doi]= all_journals
        #print(all_journals)
    
        

        




In [4]:
#print(all_journals)
print(doi_journals)

{'10.1371/journal.pone.0053940': {'Jessica M Esparza': ['PLoS ONE', 'Genetics', 'Molecular Biology of the Cell', 'Cell Motility and the Cytoskeleton'], "Eileen T. O'Toole": ['PLoS ONE', 'Genetics', 'Molecular Biology of the Cell', 'Cell Motility and the Cytoskeleton'], 'Linya Li': ['PLoS ONE', 'Genetics', 'Molecular Biology of the Cell', 'Cell Motility and the Cytoskeleton'], 'Susan K Dutcher': ['PLoS ONE', 'Genetics', 'Molecular Biology of the Cell', 'Cell Motility and the Cytoskeleton']}}


In [ ]:
#SCRAPE LINKEDIN
session = requests.Session()
username="perkinsm@usc.edu"
password="Pass123!"
login = session.get('https://www.linkedin.com/uas/login-submit')
params={'session-key' : username,
        'session-password' : password
        }
session.post('https://www.linkedin.com/uas/login-submit',data=params)


for auth in doi_first_auth_dict.values():
    
    
    